In [16]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


# Load the dataset
df = pd.read_csv('adult_with_headers.csv')  # Adjust the path to your dataset

# Display the first few rows to understand its structure
df.head()

age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country  income  
0          2174             0              40   United-States   <=50K  
1             0             0              13   United-States   <=50K  
2             0             0              40   United-States   <=50K  
3             0             0              40   United-States   <=50K  
4             0             0              40            Cuba   <=50K

In [23]:

# Basic exploration
print(df.head())  # Show first 5 rows of the dataset
print(df.info())  # Data types and missing values
print(df.describe())  # Summary statistics

# Checking for missing values
missing_values = df.isnull().sum()
print(f"Missing values:\n{missing_values}")

# Handle missing values (Simple Imputation - filling with mode for categorical and median for numerical)
df.fillna(df.mode().iloc[0], inplace=True)  # Imputing categorical variables with mode
df.fillna(df.median(), inplace=True)  # Imputing numerical variables with median





        age  workclass    fnlwgt  education  education_num  marital_status  \
0  0.301370          7  0.044302          9       0.800000               4   
1  0.452055          6  0.048238          9       0.800000               2   
2  0.287671          4  0.138113         11       0.533333               0   
3  0.493151          4  0.151068          1       0.400000               2   
4  0.150685          4  0.221488          9       0.800000               2   

   occupation  relationship    race  capital_gain  capital_loss  \
0           1             1   White       0.02174           0.0   
1           4             0   White       0.00000           0.0   
2           6             1   White       0.00000           0.0   
3           6             0   Black       0.00000           0.0   
4          10             5   Black       0.00000           0.0   

   hours_per_week  native_country  sex_ Male  income_ >50K  
0        0.397959              39       True         False  
1     

TypeError: Cannot convert [[' White' ' White' ' White' ... ' White' ' White' ' White']] to numeric

In [24]:
# Selecting numerical columns for scaling
numerical_columns = df.select_dtypes(include=[np.number]).columns

# Standard Scaling
standard_scaler = StandardScaler()
df[numerical_columns] = standard_scaler.fit_transform(df[numerical_columns])

# Min-Max Scaling
min_max_scaler = MinMaxScaler()
df[numerical_columns] = min_max_scaler.fit_transform(df[numerical_columns])



In [25]:
from sklearn.preprocessing import LabelEncoder

# Identify categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns

# Apply One-Hot Encoding to columns with <5 categories
one_hot_columns = [col for col in categorical_columns if df[col].nunique() < 5]
df = pd.get_dummies(df, columns=one_hot_columns, drop_first=True)

# Apply Label Encoding to columns with >5 categories
label_encoder = LabelEncoder()
label_columns = [col for col in categorical_columns if df[col].nunique() > 5]

for col in label_columns:
    df[col] = label_encoder.fit_transform(df[col])



In [ ]:
# Create new features
# 1. Feature: 'age_group' - Binned age into categories
df['age_group'] = pd.cut(df['age'], bins=[0, 25, 50, 75, 100], labels=['young', 'adult', 'middle-aged', 'senior'])

# 2. Feature: 'hours_per_week_range' - Categorizing working hours per week
df['hours_per_week_range'] = pd.cut(df['hours-per-week'], bins=[0, 20, 40, 60, 80], labels=['part-time', 'full-time', 'over-time', 'highly-extended'])

# Apply log transformation to skewed numerical features
df['log_hours_per_week'] = np.log1p(df['hours-per-week'])



In [ ]:
from sklearn.ensemble import IsolationForest

# Fit Isolation Forest to detect outliers
iso_forest = IsolationForest(contamination=0.05)  # Assuming 5% contamination rate
outliers = iso_forest.fit_predict(df[numerical_columns])

# Remove outliers
df_no_outliers = df[outliers == 1]  # Keep only the inliers


In [26]:
import ppscore as pps

# Calculate PPS score
pps_matrix = pps.fit(df)

# Visualize the PPS matrix
import seaborn as sns
plt.figure(figsize=(10, 8))
sns.heatmap(pps_matrix.pivot('from', 'to', 'ppscore'), annot=True, cmap='coolwarm')
plt.title('PPS Matrix')
plt.show()



ModuleNotFoundError: No module named 'ppscore'